In [151]:
import pandas as pd
import numpy as np
import geopandas
import json

from bokeh.io import show
from bokeh.io import output_notebook, show, output_file
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,BooleanFilter, GroupFilter,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          PanTool,WheelZoomTool,BoxZoomTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

from bokeh.plotting import figure, output_file, show
from bokeh.tile_providers import Vendors, get_provider
from pyproj import Proj, transform
import numpy as np
from multiprocessing import Pool


In [130]:
import tqdm
import warnings



# def lat_long_to_en(long, lat):
#     easting, northing = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), long, lat)
#     return easting, northing

# def add_features(df):
#     column_rename = {'iyear':'Year','imonth':'Month','iday':'Day','country_txt':'Country','region_txt':'Region','attacktype1_txt':'AttackType','target1':'Target','nkill':'Killed','nwound':'Wounded','summary':'Summary','gname':'Group','targtype1_txt':'Target_type','weaptype1_txt':'Weapon_type','motive':'Motive'}
#     df.rename(columns=column_rename,inplace=True)
#     df= df[['Year','Month','Day','Country','Region','city','latitude','longitude','AttackType','Killed','Wounded','Target','Summary','Group','Target_type','Weapon_type','Motive']]
#     df[['Killed', "Wounded"]] = df[['Killed', "Wounded"]].fillna(0)
#     df[['city', 'Target', 'Summary', 'Motive']] = df[['city', 'Target', 'Summary', 'Motive']].fillna("")
#     df['total_effected'] = df['Killed'] + df['Wounded']
#     df.isnull().sum()
#     with warnings.catch_warnings():
#         warnings.simplefilter("ignore")
#         df['E'], df['N'] = zip(*df.parallel_apply(lambda x: lat_long_to_en(x['longitude'], x['latitude']), axis=1))
#     return df



# terrorism_df = pd.read_csv('globalterrorismdb_0718dist.csv',encoding = "ISO-8859-1")
# terrorism_df = add_features(terrorism_df)
# terrorism_df.to_csv(index=False)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [194]:
# terrorism_df.isna().sum()
# terrorism_df['total_effected'] = terrorism_df['Killed'] + terrorism_df['Wounded']
terrorism_df = pd.read_csv('terrorism_fixed.csv')
plot_df = terrorism_df.dropna()
plot_df = plot_df[plot_df['total_effected'] > 0]
plot_df['total_effected'] = plot_df['total_effected'] 

In [195]:
# out_df = terrorism_df.to_csv('terrorism_fixed',index=False)

In [198]:

import warnings
from bokeh.palettes import brewer, inferno
from bokeh.io import curdoc
from sklearn.preprocessing import MinMaxScaler


def scale_data(df):
    df['total_effected_scaled'] = (scaler.fit_transform(df['total_effected'].values.reshape(-1,1)) *250) + 7
    return df

palette = brewer['Paired']
attack_types = list(plot_df['AttackType'].unique())
num_groups = len(attack_types)
print(num_groups)
colors = palette[num_groups]
print(colors)
color_map = {}

for i,g in enumerate(plot_df['AttackType'].unique()):
    color_map[g] = colors[i]

scaler = MinMaxScaler()
plot_df['color'] = [ color_map[x] for x in list(plot_df['AttackType'].values)]
plot_df = scale_data(plot_df)


def get_year_data(year):
    terrorism_year = plot_df[plot_df['Year'] == year]
    return terrorism_year

start_year = get_year_data(2017)
left = min(start_year['E']) - 1000
right = max(start_year['E']) + 1000
bottom = min(start_year['N']) - 1000
top = max(start_year['N']) + 1000

hover = HoverTool(tooltips = [ ('Killed','@Killed'),('Wounded', '@Wounded'), ('city','@city'), ('Target','@Target'), ('Summary','@Summary'), ('Motive','@Motive')])
p = figure(x_range=(left,right), y_range=(bottom,top),x_axis_type="mercator",y_axis_type="mercator", plot_width=1500, plot_height=1000, tools=[hover, WheelZoomTool(),BoxZoomTool(), PanTool()])
p.add_tile(get_provider(Vendors.STAMEN_TONER))
p.axis.visible=False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

def build_source(df):
    new_data = {
        'E': df['E'],
        'N': df['N'],
        'Killed': df['Killed'],
        'Wounded': df['Wounded'],
        'City': df['city'],
        'Target': df['Target'],
        'Summary': df['Summary'],
        'Motive': df['Motive'],
        'AttackType': df['AttackType'],
        'total_effected_scaled': df['total_effected_scaled'],
        'color': df['color']
    }
    return new_data


source = ColumnDataSource(data=build_source(start_year))

source0 = start_year[start_year['AttackType'] == attack_types[0]]
source1 = start_year[start_year['AttackType'] == attack_types[1]]
source2 = start_year[start_year['AttackType'] == attack_types[2]]
source3 = start_year[start_year['AttackType'] == attack_types[3]]
source4 = start_year[start_year['AttackType'] == attack_types[4]]
source5 = start_year[start_year['AttackType'] == attack_types[5]]
source6 = start_year[start_year['AttackType'] == attack_types[6]]
source7 = start_year[start_year['AttackType'] == attack_types[7]]
source8 = start_year[start_year['AttackType'] == attack_types[8]]

source_list = [source0, source1, source2, source3, source4, source5, source6, source7, source8]

for s in source_list:
p.scatter(source=source0,
     x='E', 
     y='N', 
     size = 'total_effected_scaled',
     line_color='color', 
     fill_color="color", 
     legend_group='AttackType', 
     muted_color='color', 
     muted_alpha=0.2,fill_alpha=0.45)
p.scatter(source=source1,
     x='E', 
     y='N', 
     size = 'total_effected_scaled',
     line_color='color', 
     fill_color="color", 
     legend_group='AttackType', 
     muted_color='color', 
     muted_alpha=0.2,fill_alpha=0.45)
p.scatter(source=source2,
     x='E', 
     y='N', 
     size = 'total_effected_scaled',
     line_color='color', 
     fill_color="color", 
     legend_group='AttackType', 
     muted_color='color', 
     muted_alpha=0.2,fill_alpha=0.45)
p.scatter(source=source3,
     x='E', 
     y='N', 
     size = 'total_effected_scaled',
     line_color='color', 
     fill_color="color", 
     legend_group='AttackType', 
     muted_color='color', 
     muted_alpha=0.2,fill_alpha=0.45)
p.scatter(source=source4,
     x='E', 
     y='N', 
     size = 'total_effected_scaled',
     line_color='color', 
     fill_color="color", 
     legend_group='AttackType', 
     muted_color='color', 
     muted_alpha=0.2,fill_alpha=0.45)
p.scatter(source=source5,
     x='E', 
     y='N', 
     size = 'total_effected_scaled',
     line_color='color', 
     fill_color="color", 
     legend_group='AttackType', 
     muted_color='color', 
     muted_alpha=0.2,fill_alpha=0.45)
p.scatter(source=source6,
     x='E', 
     y='N', 
     size = 'total_effected_scaled',
     line_color='color', 
     fill_color="color", 
     legend_group='AttackType', 
     muted_color='color', 
     muted_alpha=0.2,fill_alpha=0.45)
p.scatter(source=source7,
     x='E', 
     y='N', 
     size = 'total_effected_scaled',
     line_color='color', 
     fill_color="color", 
     legend_group='AttackType', 
     muted_color='color', 
     muted_alpha=0.2,fill_alpha=0.45)
p.scatter(source=source8,
     x='E', 
     y='N', 
     size = 'total_effected_scaled',
     line_color='color', 
     fill_color="color", 
     legend_group='AttackType', 
     muted_color='color', 
     muted_alpha=0.2,fill_alpha=0.45)




def update_plot(attr,old, new):
    year = slider.value
    new_df = get_year_data(year)
#     new_data = build_source(new_df)
    source0.data = build_source(new_df[new_df['AttackType'] == attack_types[0]])
    source1.data = build_source(new_df[new_df['AttackType'] == attack_types[1]])
    source2.data = build_source(new_df[new_df['AttackType'] == attack_types[2]])
    source3.data = build_source(new_df[new_df['AttackType'] == attack_types[3]])
    source4.data = build_source(new_df[new_df['AttackType'] == attack_types[4]])
    source5.data = build_source(new_df[new_df['AttackType'] == attack_types[5]])
    source6.data = build_source(new_df[new_df['AttackType'] == attack_types[6]])
    source7.data = build_source(new_df[new_df['AttackType'] == attack_types[7]])
    source8.data = build_source(new_df[new_df['AttackType'] == attack_types[8]])
#     source.data= new_data
    p.title.text = 'Total effected by terrorism in {}'.format(year)

    
p.legend.location = "bottom_left"
p.legend.click_policy="mute"
slider = Slider(title= 'Year', start=1970, end=2017, step=1, value=2017)
slider.on_change('value', update_plot)
layout = column(p, widgetbox(slider))
curdoc().add_root(layout)
show(layout)



9
('#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6')


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

